In [1]:
# 명령줄 인수 형태의 리스트 생성
args = {
    "seed": 10,
    "env": "hopper-medium-v2",
    "K": 5,
    "embed_dim": 512,
    "n_layer": 4,
    "n_head": 4,
    "activation_function": "relu",
    "dropout": 0.1,
    "eval_context_length": 5,
    "ordering": 0,
    "eval_rtg": 3600,
    "num_eval_episodes": 10,
    "init_temperature": 0.1,
    "batch_size": 256,
    "learning_rate": 1e-4,
    "weight_decay": 5e-4,
    "warmup_steps": 10000,
    "max_pretrain_iters": 1,
    "num_updates_per_pretrain_iter": 5000,
    "max_online_iters": 1500,
    "online_rtg": 7200,
    "num_online_rollouts": 1,
    "replay_size": 1000,
    "num_updates_per_online_iter": 300,
    "eval_interval": 10,
    "device": "cuda",
    "log_to_tb": True,
    "save_dir": "./exp",
    "exp_name": "default",
    "tags": "model:01"
}


# 생성된 args 리스트 확인
print(args)

from main import Experiment
import utils
import warnings
warnings.filterwarnings('ignore')
utils.set_seed_everywhere(args['seed'])
experiment = Experiment(args)
experiment()



{'seed': 10, 'env': 'hopper-medium-v2', 'K': 5, 'embed_dim': 512, 'n_layer': 4, 'n_head': 4, 'activation_function': 'relu', 'dropout': 0.1, 'eval_context_length': 5, 'ordering': 0, 'eval_rtg': 3600, 'num_eval_episodes': 10, 'init_temperature': 0.1, 'batch_size': 256, 'learning_rate': 0.0001, 'weight_decay': 0.0005, 'warmup_steps': 10000, 'max_pretrain_iters': 1, 'num_updates_per_pretrain_iter': 5000, 'max_online_iters': 1500, 'online_rtg': 7200, 'num_online_rollouts': 1, 'replay_size': 1000, 'num_updates_per_online_iter': 300, 'eval_interval': 10, 'device': 'cuda', 'log_to_tb': True, 'save_dir': './exp', 'exp_name': 'default', 'tags': 'model:01'}


/env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/env/lib/python3.8/site-packages/glfw/__init__.py:912: GLFWError: (65544) b'X11: Failed to open display :11.0'
  warnings.warn(message, GLFWError)
pybullet build time: Nov 28 2023 23:51:11
2023-12-22 05:39:02,007 - main - INFO - Starting logger
INFO:main:Starting logger
2023-12-22 05:39:02,009 - main - INFO - Starting experiment
INFO:main:Starting experiment
2023-12-22 05:39:02,010 - main - INFO - args: {'seed': 10, 'env': 'hopper-medium-v2', 'K': 5, 'embed_dim': 512, 'n_layer': 4, 'n_head': 4, 'activation_function': 'relu', 'dropout': 0.1, 'eval_context_length': 5, 'ordering': 0, 'eval_rtg': 3600, 'num_eval_episodes': 10, 'init_temperature': 0.1, 'batch_size': 256, 'learning_rate': 0.0001, 'weight_decay': 0.0005, 'warmup_steps': 10000, 'max_pretrain_i

Starting new experiment: hopper-medium-v2
2186 trajectories, 999906 timesteps found
Average return: 1422.06, std: 378.95
Max return: 3222.36, min: 315.87
Average length: 457.41, std: 110.88
Max length: 1000.00, min: 145.00
Experiment log path: ./exp/2023.12.22/053905-default


Making Eval Env.....


/env/lib/python3.8/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:569: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  (np.object, string),
/env/lib/python3.8/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:570: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  (np.bool, bool),
/env/lib/python3.8/site-packages/tensorboard/util/tensor_util.py:109: DeprecationWarning: `np.object` is a deprecated 




*** Pretrain ***


/env/lib/python3.8/site-packages/gym/spaces/box.py:84: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")
/env/lib/python3.8/site-packages/gym/core.py:172: DeprecationWarning: WARN: Function `env.seed(seed)` is marked as deprecated and will be removed in the future. Please use `env.reset(seed=seed) instead.
  deprecation(
/env/lib/python3.8/site-packages/gym/spaces/box.py:84: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")
/env/lib/python3.8/site-packages/gym/core.py:172: DeprecationWarning: WARN: Function `env.seed(seed)` is marked as deprecated and will be removed in the future. Please use `env.reset(seed=seed) instead.
  deprecation(


ValueError: The value argument must be within the support

In [ ]:
import pickle
import numpy as np
env_name = "hopper-medium-v2"
dataset_path = f"./data/{env_name}.pkl"
with open(dataset_path, "rb") as f:
    trajectories = pickle.load(f)

states, traj_lens, returns = [], [], []
for path in trajectories:
    states.append(path["observations"])
    traj_lens.append(len(path["observations"]))
    returns.append(path["rewards"].sum())
traj_lens, returns = np.array(traj_lens), np.array(returns)

# used for input normalization
states = np.concatenate(states, axis=0)
state_mean, state_std = np.mean(states, axis=0), np.std(states, axis=0) + 1e-6
num_timesteps = sum(traj_lens)

print("=" * 50)
print(f"Starting new experiment: {env_name}")
print(f"{len(traj_lens)} trajectories, {num_timesteps} timesteps found")
print(f"Average return: {np.mean(returns):.2f}, std: {np.std(returns):.2f}")
print(f"Max return: {np.max(returns):.2f}, min: {np.min(returns):.2f}")
print(f"Average length: {np.mean(traj_lens):.2f}, std: {np.std(traj_lens):.2f}")
print(f"Max length: {np.max(traj_lens):.2f}, min: {np.min(traj_lens):.2f}")
print("=" * 50)

sorted_inds = np.argsort(returns)  # lowest to highest
num_trajectories = 1
timesteps = traj_lens[sorted_inds[-1]]
ind = len(trajectories) - 2
while ind >= 0 and timesteps + traj_lens[sorted_inds[ind]] < num_timesteps:
    timesteps += traj_lens[sorted_inds[ind]]
    num_trajectories += 1
    ind -= 1
sorted_inds = sorted_inds[-num_trajectories:]
trajectories = [trajectories[ii] for ii in sorted_inds]


Starting new experiment: hopper-medium-v2
2186 trajectories, 999906 timesteps found
Average return: 1422.06, std: 378.95
Max return: 3222.36, min: 315.87
Average length: 457.41, std: 110.88
Max length: 1000.00, min: 145.00


In [ ]:
len(trajectories[0])

5

In [ ]:
trajectories[0]["observations"].shape

(161, 11)